In [19]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd, fiona
import glob
import os

In [33]:
def read_kml():
    # Load the KML file
    kml_path = 'dhaka_cIty_gis_project.kml'
    shapefile = gpd.read_file(kml_path, driver="KML", allow_unsupported_drivers=True)

    return shapefile

In [34]:
def crop_using_kml(input_file_path, output_file_path):
    with rasterio.open(input_file_path) as src:
        shapefile = read_kml()

        # Ensure shapefile has the same CRS as the raster
        shapefile = shapefile.to_crs(src.crs)

        # Convert the geometry to GeoJSON format for rasterio
        geometry = [feature["geometry"] for feature in shapefile.__geo_interface__["features"]]

        # Mask and crop the B1 raster using the shapefile geometry
        cropped_image, cropped_transform = mask(src, geometry, crop=True)

        # Update metadata to fit the cropped image
        cropped_meta = src.meta.copy()
        cropped_meta.update({
            "height": cropped_image.shape[1],
            "width": cropped_image.shape[2],
            "transform": cropped_transform
        })

    # Optionally save the cropped output
    with rasterio.open(output_file_path, 'w', **cropped_meta) as dest:
        dest.write(cropped_image)


In [35]:
ROOT = './Landsat8/'

input_path = ROOT + 'rawData/*/*.TIF'
output_path = ROOT + 'cropped/'

In [37]:
file_path_list = glob.glob(input_path)

for input_file in file_path_list:
    output_file = output_path + input_file.split('rawData/')[1]
    #print(input_file, output_file)
    crop_using_kml(input_file, output_file)

